In [1]:
import rasa_nlu
import rasa_core

In [2]:
print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))

rasa_nlu: 0.14.3 rasa_core: 0.13.2


In [3]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [4]:
# loading the nlu training samples
training_data = load_data("data/nlu_data.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("data/nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("models/current/nlu", fixed_model_name="current")

Epochs: 100%|██████████| 300/300 [00:04<00:00, 60.54it/s, loss=0.096, acc=1.000]


In [5]:
print(interpreter.parse("Chtěl bych si objednat Smažený sýr"))

{'intent': {'name': 'order_food_main_dish', 'confidence': 0.8891581892967224}, 'entities': [{'start': 23, 'end': 34, 'value': 'Smažený sýr', 'entity': 'dish', 'confidence': 0.6718105031002314, 'extractor': 'ner_crf'}], 'intent_ranking': [{'name': 'order_food_main_dish', 'confidence': 0.8891581892967224}, {'name': 'greet', 'confidence': 0.1928783655166626}, {'name': 'order_food_desert', 'confidence': 0.1203313022851944}, {'name': 'name', 'confidence': 0.09862610697746277}, {'name': 'thanks', 'confidence': 0.008810266852378845}, {'name': 'goodbye', 'confidence': 0.0}, {'name': 'order_food_drink', 'confidence': 0.0}, {'name': 'pay', 'confidence': 0.0}, {'name': 'order_food_side_dish', 'confidence': 0.0}, {'name': 'affirm', 'confidence': 0.0}], 'text': 'Chtěl bych si objednat Smažený sýr'}


In [6]:
print(interpreter)

### custom actions

In [11]:
from typing import Dict, Text, Any, List, Union

from rasa_core_sdk import ActionExecutionRejection
from rasa_core_sdk import Tracker
from rasa_core_sdk.events import SlotSet
from rasa_core_sdk.executor import CollectingDispatcher
from rasa_core_sdk.forms import FormAction, REQUESTED_SLOT

class OrderFood(FormAction):
    def name(self):

        """The first function which you should define in a form action class is
            called name which simply defines the name of the action (the same one you defined in the domain file)"""

        return "food_form"
    @staticmethod
    def required_slots(tracker:Tracker) -> List[Text]:
        """A list of required slots that the form has to fill"""

        if tracker.get_slot('drink') == 'pivo':
            return["dish", "Pilsener Urquel", "desert"]
        else:

            return ["dish", "drink", "desert"]

        """The final building block of a simple form action is a function called submit which defines 
        what should happen once all required slots are populated. 
        In a restaurant search assistant case, once all the slots are populated, 
        the assistant will execute the template utter_submit which, based on how it’s defined in a domain.yml file,
        will confirm that the assistant has all the information it needs to proceed:"""

    def submit(self):
        """defines what the form has to do after all required slots are filled"""

        dispatcher.utter_template('utter_submit', tracker)
        return []

In [8]:
endpoint = """
action_endpoint:
  url: "http://localhost:5055/webhook"
"""
#%store endpoint > endpoints.yml

You will have to start the actions server on the command line. To do that, use code block below to start the command line and run the following command: python -m rasa_core_sdk.endpoint --actions actions

##### run in console
 python -m rasa_core.train -d domain.yml -s stories.md -o  models/dialogue
 ###### you will get then:
 Persisted model to '/Users/William/starter-pack-rasa-stack/czech/data/models/dialogue'

In [ ]:
from IPython.display import IFrame
IFrame("http://localhost:8888/terminals/1", width=1000, height=500)

In [ ]:
import IPython
from IPython.display import clear_output
from rasa_core.agent import Agent
from rasa_core.interpreter import NaturalLanguageInterpreter
from rasa_core.utils import EndpointConfig
import time

from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy

messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
interpreter = NaturalLanguageInterpreter.create('/Users/William/starter-pack-rasa-stack/czech/models/current/nlu/default/current')
endpoint = EndpointConfig('http://localhost:5055/webhook')
agent = Agent.load('/Users/William/starter-pack-rasa-stack/czech/data/models/dialogue', interpreter=interpreter, action_endpoint = endpoint)
#agent = Agent.load('/Users/William/starter-pack-rasa-stack/czech/data/models/dialogue', interpreter=model_directory, action_endpoint = endpoint)

print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_text(a)
    for response in responses:
        print(response["text"])


INFO:tensorflow:Restoring parameters from /Users/William/starter-pack-rasa-stack/czech/models/current/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt
Your bot is ready to talk! Type your messages here or send 'stop'
ahoj
Vitejte None, stul pro kolik osob
jsme William
Vitejte William, stul pro kolik osob
Chtěl bych si objednat Smažený sýr
Dáte si něco k pití?
Mohl bych dostat sklenici vino
 pro vas Dáte si dezert?
Chtěl bych si objednat zmerzlina
Dobre, To je všechno
tak:
 - hlavní_jidla: Smažený sýr
 - Napoje: vino
 - Dezerty: zmerzlina
dekuji
Nashledanou
Nashledanou


In [ ]:
#!python -m rasa_core.run -d Users/William/starter-pack-rasa-stack/czech/data/models/dialogue -u models/current/nlu --endpoints data/endpoints.yml

In [ ]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

In [ ]:
from rasa_core.agent import Agent
agent = Agent.load('/Users/William/starter-pack-rasa-stack/czech/data/models/dialogue', interpreter=model_directory)

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])